<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing and Evaluating a LangChain OpenAI Agent Application</h1>

With the new OpenAI API that supports function calling, it’s never been easier to build your own agent.

In this notebook tutorial, we showcase how to write your own OpenAI agent in under 50 lines of code and use Phoenix to inspect the internals of the Agent. It is minimal, yet feature complete (with ability to carry on a conversation and use tools).

Install LangChain and other dependencies.

In [ ]:
!pip install -qq arize-phoenix langchain openai

## 2. Launch Phoenix

You can run Phoenix in the background to collect trace data emitted by any LangChain application that has been instrumented with the `OpenInferenceTracer`.

Launch Phoenix and follow the instructions in the cell output to open the Phoenix UI (the UI should be empty because we have yet to run a LangChain application).

In [ ]:
import phoenix as px

session = px.launch_app()

Let's make sure we have openAI credentials set up.

In [ ]:
from getpass import getpass
import os

import openai

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

Let's now define a callback manager that will trace through all the sub-parts of the agent. 

Note that this will not be necessary in future versions of LangChain, as we will be able to trace through the entire agent automatically via a top-level callback manager. For now however we will have to add the callback manager everywhere that we want traces.

In [ ]:
from langchain.callbacks.manager import CallbackManager
from phoenix.trace.langchain import OpenInferenceTracer

tracer = OpenInferenceTracer()
callback_manager = CallbackManager([tracer])

Let's now define the LLM model we will use for our agent.

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613", callback_manager=callback_manager)

Let's define the tools the LLM will have at its disposal. We will use the following tools:

In [ ]:
from langchain.agents import Tool
from langchain.chains import LLMMathChain

llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
# Let's give the LLM access to math tools
tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
        callback_manager=callback_manager,
    ),
]

Let's setup the Prompt Template. This will inform how the agent will respond to queries.

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

## Initialize the OpenAI Agent

Now, we define our agent that’s capable answering questions and calling tools.

The meat of the agent logic breaks down into three steps:

- Call OpenAI to decide which tool (if any) to call and with what arguments.

- Call the tool with the arguments to obtain an output

- Call OpenAI to synthesize a response from the conversation context and the tool output.

Let's initialize the agent. Notice that we are using the `OpenInferenceTracer` to trace the agent's inferences as traces.


In [ ]:
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613", callback_manager=callback_manager)
agent_executor = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True, callback_manager=callback_manager
)

Let's now chat with our agent! Let's add the tracer to get the details of the execution into phoenix.

In [ ]:
response = agent_executor.invoke({"input": "What is 47 raised to the 5th power?"})
response

Let's chat with our agent a few more times. This time with some follow-up questions.

In [ ]:
queries = [
    "What is (121 * 3) + 42?",
    "what is 3 * 3?",
    "what is 4 * 4?",
    "what is 75 * (3 + 4)?",
    "what is 23 times 87",
]

for query in queries:
    print(f"> {query}")
    response = agent_executor.invoke({"input": query})
    print(response)
    print("---")

Open the `session.url` in your browser to take a look at the traces in Phoenix. Note that LLM spans contain the OpenAI function calls, and that we can inspect what tool the LLM picked based on the queries.

To learn more about function calling, check out the [OpenAI API docs](https://openai.com/blog/function-calling-and-other-api-updates).


In [ ]:
print(f"View the traces in phoenix: {session.url}")

We can also inspect the agent's chat history as a dataframe.

In [ ]:
ds = px.TraceDataset.from_spans(list(tracer.get_spans()))
ds.dataframe.head()

If you would like you can write the conversations to a file for later use.

In [ ]:
# Dump the contents to a file for safe keeping
export_trace = True
if export_trace:
    from phoenix.trace.span_json_encoder import spans_to_jsonl

    with open("trace.jsonl", "w") as f:
        f.write(spans_to_jsonl(tracer.get_spans()))